In [1]:
import pandas as pd
from pytz import timezone
from datetime import datetime, timedelta

# Define the data
store_id = {"store_id": 1481966498820158979}

store_timezone_data = {"store_id": 1481966498820158979, "timezone_str": "Asia/Beirut"}

# Convert store hours data to pandas dataframe
df_hours = pd.read_csv("store_hours_data.csv")
# Convert timestamp_utc to datetime format
df_status = pd.read_csv("store_status_history.csv")
df_status['timestamp_utc'] = pd.to_datetime(df_status['timestamp_utc'])
# Set store timezone
store_timezone = timezone(store_timezone_data["timezone_str"])

# Calculate uptime/downtime for the last hour/day/week
now_utc = datetime.utcnow()
now_local = now_utc.astimezone(store_timezone)
one_hour_ago_local = now_local - timedelta(hours=1)
one_day_ago_local = now_local - timedelta(days=1)
one_week_ago_local = now_local - timedelta(weeks=1)

In [2]:
uptime_last_hour = df_hours.loc[(df_hours['start_time_local'] <= one_hour_ago_local.strftime('%H:%M:%S')) &
                                (df_hours['end_time_local'] >= now_local.strftime('%H:%M:%S'))]
uptime_last_day = df_hours.loc[(df_hours['start_time_local'] <= one_day_ago_local.strftime('%H:%M:%S')) &
                               (df_hours['end_time_local'] >= now_local.strftime('%H:%M:%S'))]
uptime_last_week = df_hours.loc[(df_hours['start_time_local'] <= one_week_ago_local.strftime('%H:%M:%S')) &
                                (df_hours['end_time_local'] >= now_local.strftime('%H:%M:%S'))]

downtime_last_hour = timedelta(hours=1) - uptime_last_hour['end_time_local'].apply(lambda x: datetime.strptime(x, '%H:%M:%S') - datetime.strptime(one_hour_ago_local.strftime('%H:%M:%S'), '%H:%M:%S')).sum()

print(uptime_last_week)

                  store_id  day start_time_local end_time_local
10749  6125592260000297762    0         00:00:00       07:30:00
10750  5393646129225600094    1         06:30:00       07:30:00
10751  5393646129225600094    4         06:30:00       07:30:00
10752  5393646129225600094    2         06:30:00       07:30:00
10753  5393646129225600094    0         06:30:00       07:30:00
...                    ...  ...              ...            ...
86193  1774428314218854399    4         00:00:00       23:59:59
86194  1774428314218854399    6         00:00:00       23:59:59
86195  1774428314218854399    3         00:00:00       23:59:59
86196  1774428314218854399    2         00:00:00       23:59:59
86197  1774428314218854399    1         00:00:00       23:59:59

[25701 rows x 4 columns]


In [8]:
uptime_last_hour

store_id  day start_time_local end_time_local
10749  6125592260000297762    0         00:00:00       07:30:00
10755  7185390482379651317    0         00:00:00       07:30:00
10756  8556921637233741846    6         00:00:00       08:00:00
10757  1689499139337485669    0         00:00:00       08:00:00
10758  1689499139337485669    6         00:00:00       08:00:00
...                    ...  ...              ...            ...
86193  1774428314218854399    4         00:00:00       23:59:59
86194  1774428314218854399    6         00:00:00       23:59:59
86195  1774428314218854399    3         00:00:00       23:59:59
86196  1774428314218854399    2         00:00:00       23:59:59
86197  1774428314218854399    1         00:00:00       23:59:59

[19709 rows x 4 columns]

In [10]:
import pandas as pd
from pytz import timezone
from datetime import datetime, timedelta

# Define the data
store_id = {"store_id": 1481966498820158979}

store_hours_data = [{'store_id': 1481966498820158979,
  'day': 4,
  'start_time_local': '00:00:00',
  'end_time_local': '00:10:00'},
 {'store_id': 1481966498820158979,
  'day': 2,
  'start_time_local': '00:00:00',
  'end_time_local': '00:10:00'},
 {'store_id': 1481966498820158979,
  'day': 0,
  'start_time_local': '00:00:00',
  'end_time_local': '00:10:00'},]

store_status_history = [{'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': '2023-01-20 05:37:32.091231 UTC'},
 {'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': '2023-01-24 15:10:01.222494 UTC'},
 {'store_id': 1481966498820158979,
  'status': 'active',
  'timestamp_utc': '2023-01-19 00:35:26.427784 UTC'}]

store_timezone_data = {"store_id": 1481966498820158979, "timezone_str": "Asia/Beirut"}

# Convert hours and minutes to timedelta objects
def to_timedelta(time_str):
    hours, minutes, seconds = time_str.split(':')
    return timedelta(hours=int(hours), minutes=int(minutes), seconds=int(seconds))

# Calculate the timezone offset
timezone_offset = timezone(store_timezone_data["timezone_str"]).utcoffset(datetime.utcnow()).total_seconds() / 60

# Get the current time in the store's timezone
current_time = datetime.utcnow() + timedelta(minutes=timezone_offset)

# Get the start and end time for the last hour
last_hour_start = current_time - timedelta(hours=1)
last_hour_end = current_time

# Convert the start and end time to the store's timezone
last_hour_start_local = timezone(store_timezone_data["timezone_str"]).localize(last_hour_start).strftime("%Y-%m-%d %H:%M:%S")
last_hour_end_local = timezone(store_timezone_data["timezone_str"]).localize(last_hour_end).strftime("%Y-%m-%d %H:%M:%S")

# Filter the store hours data to get the hours for the last hour
last_hour_hours = [to_timedelta(hour["end_time_local"]) - to_timedelta(hour["start_time_local"]) for hour in store_hours_data if hour["day"] == last_hour_start.weekday() and hour["end_time_local"] > last_hour_start_local and hour["start_time_local"] < last_hour_end_local]

# Calculate the uptime for the last hour in minutes
uptime_last_hour = int(sum(last_hour_hours, timedelta()).total_seconds() / 60)

In [18]:
import pymongo
client = pymongo.MongoClient("mongodb://localhost:27017/")

In [19]:
db = client["store"]

In [23]:
store_hours_data_ = db["store_hours_data"]

In [24]:
store_status_history_ = db['store_status_history']

In [26]:
import pytz
from datetime import datetime, timedelta

# Define the data
store_id = {"store_id": 1481966498820158979}

store_hours_data = list(store_hours_data_.find(store_id))

store_status_history = list(store_status_history_.find(store_id))
print(len(store_status_history))

store_timezone_data = {"store_id": 1481966498820158979, "timezone_str": "Asia/Beirut"}

# Convert the start and end times to datetime objects in the store's timezone
store_timezone = pytz.timezone(store_timezone_data["timezone_str"])
start_time = datetime.strptime(store_hours_data[0]["start_time_local"], '%H:%M:%S')
end_time = datetime.strptime(store_hours_data[0]["end_time_local"], '%H:%M:%S')
start_time = store_timezone.localize(datetime.now().replace(hour=start_time.hour, minute=start_time.minute, second=start_time.second))
end_time = store_timezone.localize(datetime.now().replace(hour=end_time.hour, minute=end_time.minute, second=end_time.second))

# Convert the timestamp to a datetime object in the UTC timezone
timestamp_utc = datetime.strptime(store_status_history[0]["timestamp_utc"], '%Y-%m-%d %H:%M:%S.%f UTC').replace(tzinfo=pytz.utc)

# Calculate the time difference between the start time and the timestamp
uptime_timedelta = timestamp_utc - start_time

# Convert the time difference to minutes
uptime_last_hour = int(uptime_timedelta.total_seconds() / 60)

print("Uptime last hour:", uptime_last_hour, "minutes")

113
Uptime last hour: -87382 minutes


In [32]:
import pytz
from datetime import datetime, timedelta

# Define the data
store_id = {"store_id": 8419537941919820732}

store_hours_data = list(store_hours_data_.find(store_id))

store_status_history = list(store_status_history_.find(store_id))

store_timezone_data = {"store_id": 8419537941919820732, "timezone_str": "Asia/Beirut"}

# Convert the start and end times to datetime objects in the store's timezone
store_timezone = pytz.timezone(store_timezone_data["timezone_str"])
start_time = datetime.strptime(store_hours_data[0]["start_time_local"], '%H:%M:%S')
end_time = datetime.strptime(store_hours_data[0]["end_time_local"], '%H:%M:%S')
start_time = store_timezone.localize(datetime.now().replace(hour=start_time.hour, minute=start_time.minute, second=start_time.second))
end_time = store_timezone.localize(datetime.now().replace(hour=end_time.hour, minute=end_time.minute, second=end_time.second))

# Convert the timestamp to a datetime object in the UTC timezone
timestamp_utc = datetime.strptime(store_status_history[0]["timestamp_utc"], '%Y-%m-%d %H:%M:%S.%f UTC').replace(tzinfo=pytz.utc)

# Calculate the time difference between the start time and the timestamp
if timestamp_utc > start_time:
    uptime_timedelta = timestamp_utc - start_time
    uptime_last_hour = int(uptime_timedelta.total_seconds() / 60)
    print("Uptime last hour:", uptime_last_hour, "minutes")
else:
    print("Store is not active during the past hour.")
    
print("Uptime last hour:", uptime_last_hour, "minutes")

IndexError: list index out of range

In [31]:
store_status_history

[{'_id': ObjectId('641a84006f22a99af44c1c10'),
  'store_id': 8419537941919820732,
  'status': 'active',
  'timestamp_utc': '2023-01-22 12:09:39.388884 UTC'},
 {'_id': ObjectId('641a84006f22a99af44c3ea8'),
  'store_id': 8419537941919820732,
  'status': 'active',
  'timestamp_utc': '2023-01-24 09:00:43.697516 UTC'},
 {'_id': ObjectId('641a84006f22a99af44c68fb'),
  'store_id': 8419537941919820732,
  'status': 'active',
  'timestamp_utc': '2023-01-19 09:03:29.188572 UTC'},
 {'_id': ObjectId('641a84006f22a99af44ca01c'),
  'store_id': 8419537941919820732,
  'status': 'active',
  'timestamp_utc': '2023-01-25 13:03:12.143319 UTC'},
 {'_id': ObjectId('641a84006f22a99af44cf2d1'),
  'store_id': 8419537941919820732,
  'status': 'active',
  'timestamp_utc': '2023-01-24 05:54:30.441113 UTC'},
 {'_id': ObjectId('641a84006f22a99af44d1c59'),
  'store_id': 8419537941919820732,
  'status': 'active',
  'timestamp_utc': '2023-01-19 12:04:39.681565 UTC'},
 {'_id': ObjectId('641a84006f22a99af44d2123'),
  'st

In [36]:
import pytz
from datetime import datetime, timedelta

# Define the data
store_id = {"store_id": 8419537941919820732}

store_hours_data = list(store_hours_data_.find(store_id))

store_status_history = list(store_status_history_.find(store_id))

store_timezone_data = {"store_id": 8419537941919820732, "timezone_str": "Asia/Beirut"}

# Convert the start and end times to datetime objects in the store's timezone
if store_hours_data:
    start_time = datetime.strptime(store_hours_data[0]["start_time_local"], '%H:%M:%S')
    end_time = datetime.strptime(store_hours_data[0]["end_time_local"], '%H:%M:%S')
else:
    start_time = datetime.min.time()
    end_time = datetime.max.time()

store_timezone = pytz.timezone(store_timezone_data["timezone_str"])
start_time = store_timezone.localize(datetime.now().replace(hour=start_time.hour, minute=start_time.minute, second=start_time.second))
end_time = store_timezone.localize(datetime.now().replace(hour=end_time.hour, minute=end_time.minute, second=end_time.second))

# Convert the timestamp to a datetime object in the UTC timezone
timestamp_utc = datetime.strptime(store_status_history[0]["timestamp_utc"], '%Y-%m-%d %H:%M:%S.%f UTC').replace(tzinfo=pytz.utc)

# Calculate the time difference between the start time and the timestamp
if timestamp_utc > start_time:
    uptime_timedelta = timestamp_utc - start_time
    uptime_last_hour = int(uptime_timedelta.total_seconds() / 60)
    print("Uptime last hour:", uptime_last_hour, "minutes")
else:
    print("Store is not active during the past hour.")
print("Uptime last hour:", uptime_last_hour if uptime_last_hour >=0 else 0, "minutes")

Store is not active during the past hour.
Uptime last hour: 0 minutes
